# Evaluation

Evaluation is a crucial part in all machine learning tasks. In this notebook, we will walk through the whole pipeline of evaluating the performance of an embedding model on [MS Marco](https://microsoft.github.io/msmarco/), and use three metrics to show its performance.

## Step 0: Setup

Install the dependencies in the environment.

In [ ]:
# !conda install pytorch::faiss-gpu

## Step 1: Load Dataset

First, download the queries and MS Marco from Huggingface Dataset

In [1]:
from datasets import load_dataset
import numpy as np

In [2]:
query_dataset = load_dataset(
    "json", data_files="ft_data/eval_dataset.json", split="train"
)
query_dataset

Dataset({
    features: ['query', 'pos', 'id', 'prompt'],
    num_rows: 1396
})

In [3]:
query_dataset[0]

{'query': 'what are prime costs',
 'pos': ['- Prime cost basically is the cost of direct labor and cost of direct material; whereas conversion costs is Overhead cost and direct labor cost.'],
 'id': '4142',
 'prompt': 'Represent this sentence for searching relevant passages: '}

In [4]:
corpus_dataset = load_dataset("json", data_files="ft_data/corpus.jsonl", split="train")
corpus_dataset

Dataset({
    features: ['text'],
    num_rows: 7437
})

If you have GPU and would like to try out the full evaluation of MS Marco, uncomment and run the following cell:

## Step 2: Embedding

Choose the embedding model that we would like to evaluate, and encode the corpus to embeddings.

In [5]:
from FlagEmbedding import FlagModel

from FlagEmbedding.inference.embedder.encoder_only.base import BaseEmbedder

bert_embedding = BaseEmbedder("BAAI/bge-m3")

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

pytorch_model.bin:   2%|1         | 41.9M/2.27G [00:00<?, ?B/s]

In [17]:
bert_embedding.model

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(8194, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elem

In [6]:
corpus_dataset[0]

{'text': 'William Bradford, Governor of Plymouth Colony http://en.wikipedia.org/wiki/William_Bradford_(Plymouth_Colony_governor) William Bradford (c.1590 â\x80\x93 1657) was an English Separatist leader in Leiden, Holland and in Plymouth Colony was a signatory to the Mayflower Compact. He served as Plymouth Colony Governor five times covering about thirty years between 1621 and 1657.'}

In [7]:
# get the embedding of the corpus
corpus_embeddings = bert_embedding.encode(corpus_dataset["text"])

print("shape of the corpus embeddings:", corpus_embeddings.shape)
print("data type of the embeddings: ", corpus_embeddings.dtype)

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

pre tokenize: 100%|██████████| 30/30 [00:00<00:00, 109.17it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 30/30 [00:05<00:00,  5.67it/s]


shape of the corpus embeddings: (7437, 1024)
data type of the embeddings:  float16


## Step 3: Indexing

We use the index_factory() functions to create a Faiss index we want:

- The first argument `dim` is the dimension of the vector space, in this case is 768 if you're using bge-base-en-v1.5.

- The second argument `'Flat'` makes the index do exhaustive search.

- The thrid argument `faiss.METRIC_INNER_PRODUCT` tells the index to use inner product as the distance metric.

In [8]:
import faiss

# get the length of our embedding vectors, vectors by bge-base-en-v1.5 have length 768
dim = corpus_embeddings.shape[-1]

# create the faiss index and store the corpus embeddings into the vector space
index = faiss.index_factory(dim, 'Flat', faiss.METRIC_INNER_PRODUCT)
corpus_embeddings = corpus_embeddings.astype(np.float32)
# train and add the embeddings to the index
index.train(corpus_embeddings)
index.add(corpus_embeddings)

print(f"total number of vectors: {index.ntotal}")

total number of vectors: 7437


Since the embedding process is time consuming, it's a good choice to save the index for reproduction or other experiments.

Uncomment the following lines to save the index.

In [8]:
# path = "./index.bin"
# faiss.write_index(index, path)

If you already have stored index in your local directory, you can load it by:

In [ ]:
# index = faiss.read_index("./index.bin")

## Step 4: Retrieval

Get the embeddings of all the queries, and get their corresponding ground truth answers for evaluation.

In [9]:
query_dataset, corpus_dataset

(Dataset({
     features: ['query', 'pos', 'id', 'prompt'],
     num_rows: 1396
 }),
 Dataset({
     features: ['text'],
     num_rows: 7437
 }))

In [10]:
queries = query_dataset["query"]
corpus = corpus_dataset["text"]

query_embeddings = bert_embedding.encode_queries(queries)
# ground_truths = [corpus[int(id)] for id in query_dataset["id"]]
ground_truths = query_dataset["pos"]
corpus = np.asarray(corpus)

Inference Embeddings: 100%|██████████| 6/6 [00:00<00:00, 21.75it/s]


Use the faiss index to search top $k$ answers of each query.

In [11]:
from tqdm import tqdm

res_scores, res_ids, res_text = [], [], []
query_size = len(query_embeddings)
batch_size = 256
# The cutoffs we will use during evaluation, and set k to be the maximum of the cutoffs.
cut_offs = [1, 10]
k = max(cut_offs)

for i in tqdm(range(0, query_size, batch_size), desc="Searching"):
    q_embedding = query_embeddings[i : min(i + batch_size, query_size)].astype(
        np.float32
    )
    # search the top k answers for each of the queries
    score, idx = index.search(q_embedding, k=k)
    res_scores += list(score)
    res_ids += list(idx)
    res_text += list(corpus[idx])

Searching: 100%|██████████| 6/6 [00:00<00:00, 112.82it/s]


## Step 5: Evaluate

### 5.1 Recall

Recall represents the model's capability of correctly predicting positive instances from all the actual positive samples in the dataset.

$$\textbf{Recall}=\frac{\text{True Positives}}{\text{True Positives}+\text{False Negatives}}$$

Recall is useful when the cost of false negatives is high. In other words, we are trying to find all objects of the positive class, even if this results in some false positives. This attribute makes recall a useful metric for text retrieval tasks.

In [12]:
def calc_recall(preds, truths, cutoffs):
    recalls = np.zeros(len(cutoffs))
    for text, truth in zip(preds, truths):
        for i, c in enumerate(cutoffs):
            recall = np.intersect1d(truth, text[:c])
            recall_num = min(len(recall), 1)
            recalls[i] += recall_num / max(min(c, len(truth)), 1)
            # recalls[i] += len(recall) / max(min(c, len(truth)), 1)
    recalls /= len(preds)
    return recalls


recalls = calc_recall(res_text, ground_truths, cut_offs)
for i, c in enumerate(cut_offs):
    print(f"recall@{c}: {recalls[i]}")

recall@1: 0.9484240687679083
recall@10: 0.9695558739255016


### 5.2 MRR

Mean Reciprocal Rank ([MRR](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)) is a widely used metric in information retrieval to evaluate the effectiveness of a system. It measures the rank position of the first relevant result in a list of search results.

$$MRR=\frac{1}{|Q|}\sum_{i=1}^{|Q|}\frac{1}{rank_i}$$

where 
- $|Q|$ is the total number of queries.
- $rank_i$ is the rank position of the first relevant document of the i-th query.

In [13]:
def MRR(preds, truth, cutoffs):
    mrr = [0 for _ in range(len(cutoffs))]
    for pred, t in zip(preds, truth):
        for i, c in enumerate(cutoffs):
            for j, p in enumerate(pred):
                if j < c and p in t:
                    mrr[i] += 1/(j+1)
                    break
    mrr = [k/len(preds) for k in mrr]
    return mrr

In [14]:
mrr = MRR(res_text, ground_truths, cut_offs)
for i, c in enumerate(cut_offs):
    print(f"MRR@{c}: {mrr[i]}")

MRR@1: 0.9484240687679083
MRR@10: 0.9703762450538954


### 5.3 nDCG

Normalized Discounted cumulative gain (nDCG) measures the quality of a ranked list of search results by considering both the position of the relevant documents and their graded relevance scores. The calculation of nDCG involves two main steps:

1. Discounted cumulative gain (DCG) measures the ranking quality in retrieval tasks.

$$DCG_p=\sum_{i=1}^p\frac{2^{rel_i}-1}{\log_2(i+1)}$$

2. Normalized by ideal DCG to make it comparable across queries.
$$nDCG_p=\frac{DCG_p}{IDCG_p}$$
where $IDCG$ is the maximum possible DCG for a given set of documents, assuming they are perfectly ranked in order of relevance.

In [15]:
pred_hard_encodings = []
for pred, label in zip(res_text, ground_truths):
    pred_hard_encoding = list(np.isin(pred, label).astype(int))
    pred_hard_encodings.append(pred_hard_encoding)

In [16]:
from sklearn.metrics import ndcg_score

for i, c in enumerate(cut_offs):
    nDCG = ndcg_score(pred_hard_encodings, res_scores, k=c)
    print(f"nDCG@{c}: {nDCG}")

nDCG@1: 0.9484240687679083
nDCG@10: 0.9769695512309843


Congrats! You have walked through a full pipeline of evaluating an embedding model. Feel free to play with different datasets and models!